##### Copyright 2025 Google LLC.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Self-Improving Prompts

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/prompting/Self_Improving_Prompts.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

<table>
  <tr>
    <td bgcolor="#d7e6ff">
      <a href="https://github.com/Karanjot786" target="_blank" title="View profile on GitHub">
        <img src="https://github.com/Karanjot786.png?size=100"
             alt="Karanjot786's GitHub avatar"
             width="100"
             height="100">
      </a>
    </td>
    <td bgcolor="#d7e6ff">
      <h2><font color='black'>This notebook was contributed by <a href="https://github.com/Karanjot786" target="_blank"><font color='#217bfe'><strong>Karanjot786</strong></font></a>.</font></h2>
      <h5><font color='black'>See <a href="https://github.com/Karanjot786" target="_blank"><font color="#078efb"><strong>Karanjot786</strong></font></a> other notebooks <a href="https://github.com/search?q=repo%3Agoogle-gemini%2Fcookbook%20%22Karanjot786%22&type=code" target="_blank"><font color="#078efb">here</font></a>.</h5></font><br>
      <font color='black'><small><em>Have a cool Gemini example? Feel free to <a href="https://github.com/google-gemini/cookbook/blob/main/CONTRIBUTING.md" target="_blank"><font color="#078efb">share it too</font></a>!</em></small></font>
    </td>
  </tr>
</table>

Prompt engineering often involves trial and error. You write a prompt, check the output, then manually tweak the prompt until it works.

This notebook shows a different approach: let Gemini improve the prompt for you.

The self-improving prompts pattern works like this:
1. Run an initial prompt
2. Have Gemini critique the output
3. Have Gemini rewrite the prompt based on the critique
4. Run the improved prompt
5. Repeat until satisfied

This technique is useful when:
- You have a rough idea but need to refine it
- You want to understand what makes a prompt effective
- You need to optimize prompts at scale

## Setup

In [2]:
%pip install -U -q "google-genai>=1.0.0"

Note: you may need to restart the kernel to use updated packages.


Select the model you want to use:

In [3]:
MODEL_ID = "gemini-3-flash-preview" # @param ["gemini-2.5-flash-lite", "gemini-2.5-flash", "gemini-2.5-pro", "gemini-3-flash-preview", "gemini-3-pro-preview"] {"allow-input":true, isTemplate: true}

### Configure your API key

To run the following cell, your API key must be stored in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [5]:
from google.colab import userdata
from google import genai

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

## Step 1: Start with a weak prompt

Start with a simple prompt that produces okay results. The goal is to see how much Gemini can improve it.

In [6]:
weak_prompt = """
Summarize this text:

The quick brown fox jumps over the lazy dog. The dog was not really lazy, 
it was just tired from playing all day. The fox was looking for food and 
saw the dog sleeping. The fox jumped over to get to the other side of the 
yard where there was a chicken coop. The chickens were scared but the fox 
did not catch any of them. The farmer heard the noise and came out. The 
fox ran away. The dog woke up and barked at the fox.
"""

## Step 2: Run the weak prompt

In [7]:
initial_response = client.models.generate_content(
    model=MODEL_ID,
    contents=weak_prompt
)

initial_output = initial_response.text
print("Initial output:")
print(initial_output)

Initial output:
A hungry fox jumped over a tired, sleeping dog to reach a chicken coop. However, the fox failed to catch any chickens because the farmer was alerted by the noise and chased him away.


## Step 3: Have Gemini critique the prompt

Ask Gemini to analyze what is missing or unclear in the original prompt.

In [8]:
critique_prompt = f"""
You are a prompt engineering expert.

Here is a prompt:
---
{weak_prompt}
---

Here is the output it produced:
---
{initial_output}
---

Critique this prompt. Identify:
1. What is missing from the prompt
2. What is unclear or ambiguous
3. How the output could be improved

Be specific and concise. Focus on actionable improvements.
"""

critique_response = client.models.generate_content(
    model=MODEL_ID,
    contents=critique_prompt
)

critique = critique_response.text
print("Critique:")
print(critique)

Critique:
As a prompt engineering expert, here is a critique of your prompt:

### 1. What is missing from the prompt
*   **Constraints:** There are no limits on length (e.g., "in under 20 words," "in one sentence," or "in three bullet points").
*   **Format:** The prompt doesn’t specify a desired structure (e.g., a formal paragraph, a narrative hook, or a TL;DR).
*   **Persona/Audience:** There is no context regarding who the summary is for (e.g., "summarize for a child" vs. "summarize for a data analyst").
*   **Tone:** The prompt lacks stylistic guidance (e.g., "playful," "objective," or "suspenseful").

### 2. What is unclear or ambiguous
*   **The "Summarize" Instruction:** The word "summarize" is a "weak" verb because it is subjective. Without specific parameters, the AI guesses the level of detail. It is unclear whether you want a high-level theme (a fox's failed hunt) or a chronological retelling of events.
*   **Information Hierarchy:** The prompt doesn't state what to prioriti

## Step 4: Have Gemini rewrite the prompt

Based on the critique, ask Gemini to write a better version of the prompt.

In [9]:
rewrite_prompt = f"""
Based on the following critique, rewrite the original prompt to produce 
a better output.

Original prompt:
---
{weak_prompt}
---

Critique:
---
{critique}
---

Write the improved prompt. Output only the new prompt text, nothing else.
"""

rewrite_response = client.models.generate_content(
    model=MODEL_ID,
    contents=rewrite_prompt
)

improved_prompt = rewrite_response.text
print("Improved prompt:")
print(improved_prompt)

Improved prompt:
Role: Children's story editor. 
Task: Summarize the provided text into a single, engaging sentence of no more than 25 words. 
Tone: Whimsical and narrative.
Constraint: Ensure you mention the fox’s goal and the ultimate outcome for both the fox and the dog.

Text: 
The quick brown fox jumps over the lazy dog. The dog was not really lazy, it was just tired from playing all day. The fox was looking for food and saw the dog sleeping. The fox jumped over to get to the other side of the yard where there was a chicken coop. The chickens were scared but the fox did not catch any of them. The farmer heard the noise and came out. The fox ran away. The dog woke up and barked at the fox.


## Step 5: Run the improved prompt

In [10]:
improved_response = client.models.generate_content(
    model=MODEL_ID,
    contents=improved_prompt
)

improved_output = improved_response.text
print("Improved output:")
print(improved_output)

Improved output:
To reach the chickens, the fox vaulted over the dozing dog, but fled empty-pawed once the farmer appeared and the pup woke to bark.


## Step 6: Compare before and after

In [11]:
print("=" * 50)
print("BEFORE (weak prompt):")
print(initial_output)
print("=" * 50)
print("AFTER (improved prompt):")
print(improved_output)
print("=" * 50)

BEFORE (weak prompt):
A hungry fox jumped over a tired, sleeping dog to reach a chicken coop. However, the fox failed to catch any chickens because the farmer was alerted by the noise and chased him away.
AFTER (improved prompt):
To reach the chickens, the fox vaulted over the dozing dog, but fled empty-pawed once the farmer appeared and the pup woke to bark.


## Create a reusable function

Wrap the self-improvement loop into a function you can use with any prompt.

In [12]:
def improve_prompt(original_prompt, iterations=2, verbose=True):
    """
    Iteratively improve a prompt using Gemini self-critique.

    Args:
        original_prompt: The initial prompt to improve
        iterations: Number of improvement cycles (default: 2)
        verbose: Print progress (default: True)

    Returns:
        Tuple of (final_prompt, final_output)
    """
    current_prompt = original_prompt

    for i in range(iterations):
        # Run current prompt
        output = client.models.generate_content(
            model=MODEL_ID,
            contents=current_prompt
        ).text

        # Critique
        critique = client.models.generate_content(
            model=MODEL_ID,
            contents=f"""
            Critique this prompt and its output. Be specific and actionable.

            Prompt:
            {current_prompt}

            Output:
            {output}
            """
        ).text

        # Rewrite
        current_prompt = client.models.generate_content(
            model=MODEL_ID,
            contents=f"""
            Rewrite this prompt based on the critique.
            Output only the new prompt text.

            Original prompt:
            {current_prompt}

            Critique:
            {critique}
            """
        ).text

        if verbose:
            print(f"Iteration {i + 1} complete")

    # Final run
    final_output = client.models.generate_content(
        model=MODEL_ID,
        contents=current_prompt
    ).text

    return current_prompt, final_output

## Test the function with a new example

In [13]:
test_prompt = "Write a poem about coding."

final_prompt, final_output = improve_prompt(test_prompt, iterations=2)

print("\nFINAL PROMPT:")
print(final_prompt)
print("\nFINAL OUTPUT:")
print(final_output)

Iteration 1 complete
Iteration 2 complete

FINAL PROMPT:
Write a Petrarchan sonnet in strict iambic pentameter with a weary, introspective tone about the quiet satisfaction of refactoring a bloated legacy codebase into clean, PEP 8-compliant Python. Avoid clichés like "dark rooms" or "missing semicolons"; instead, use grounded metaphors of architectural restoration or pruning a wild garden—stripping the scaffolding from brittle functions or thinning out overgrown, deep indentations. Focus on the physical relief of deleting redundant lines and the mental clarity provided by specific Pythonic nuances, such as replacing tangled globals with pure-built functions, the order of clean imports, and the serene breath of a concise docstring.

FINAL OUTPUT:
I peel the brittle layers from the frame,
Where nested loops once choked the morning light.
This tangled sprawl, a vast and jagged sight,
Had buried every purpose in its name.
I strip the scaffold from the ancient shame
Of global states that r

## Next steps

### Related notebooks

- [Chain of thought prompting](./Chain_of_thought_prompting.ipynb)
- [Self-ask prompting](./Self_ask_prompting.ipynb)
- [Few-shot prompting](./Few_shot_prompting.ipynb)

### Ideas to extend this pattern

- Add a scoring function to measure improvement
- Use multiple Gemini models (one for critique, one for rewriting)
- Store improvement history for analysis
- Combine with few-shot examples for domain-specific optimization